#修改前psqi量表计算

In [1]:
import openpyxl
import re
from datetime import datetime
from datetime import time
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager
import pandas as pd
from openpyxl import Workbook

In [2]:
def calculate_score_b_1(score):
    '''
    根据从上床到入睡所需的时间计算评分：
    - 如果时间小于或等于15分钟，则评分为0分
    - 如果时间在16到30分钟之间，则评分为1分
    - 如果时间在31到60分钟之间，则评分为2分
    - 如果时间大于或等于60分钟，则评分为3分
    '''
    if score <= 15:
        return 0
    elif 16 <= score <= 30:
        return 1
    elif 31 <= score <= 60:
        return 2
    elif score >= 60:
        return 3


In [3]:
def calculate_score_b_2(score):
    '''
    根据入睡困难的频率计算评分：
    - 如果无入睡困难，则评分为0分
    - 如果每周少于1次入睡困难，则评分为1分
    - 如果每周1到2次入睡困难，则评分为2分
    - 如果每周3次及以上入睡困难，则评分为3分
    '''
    if score == 0:
        return 0
    elif 1 <= score <= 2:
        return 1
    elif 3 <= score <= 4:
        return 2
    elif score >= 5:
        return 3


In [4]:
def calculate_score_c(score):
    '''
    根据每夜通常实际睡眠时间计算评分：
    - 如果实际睡眠时间大于7小时，则评分为0分
    - 如果实际睡眠时间在6到7小时之间，则评分为1分
    - 如果实际睡眠时间在5到6小时之间，则评分为2分
    - 如果实际睡眠时间小于5小时，则评分为3分
    '''
    if score < 5:
        return 3
    elif 5 <= score <= 6:
        return 2
    elif 6 < score <= 7:
        return 1
    elif score > 7:
        return 0


In [5]:
def time_to_decimal(time_str):
    '''
    将datetime.time对象或者字符串时间对象转换为小时的小数表示。
    
    参数:
    - time_str: datetime.time对象或时间字符串（格式为'HH:MM:SS'或'HH:MM'）

    返回:
    - 小数表示的小时数

    异常:
    - ValueError: 如果时间字符串无效
    - TypeError: 如果输入类型不是字符串或datetime.time对象
    '''
    if isinstance(time_str, time):
        # 如果是 datetime.time 对象，直接提取小时、分钟和秒
        hours = time_str.hour
        minutes = time_str.minute
        seconds = time_str.second
    elif isinstance(time_str, str):
        # 如果是字符串，使用正则表达式进行匹配
        match = re.match(r'(\d+)[\uff1a:](\d+)(?:[\uff1a:](\d+))?', time_str)
        if match:
            hours, minutes, seconds = map(lambda x: int(x) if x else 0, match.groups())
        else:
            raise ValueError(f"无效的时间字符串: {time_str}")
    else:
        raise TypeError(f"预期字符串或字节类型对象，但接收到 {type(time_str).__name__}")

    return hours + minutes / 60 + seconds / 3600


In [6]:
def calculate_score_d(score):
    '''
    根据睡眠效率计算评分：
    - 如果睡眠效率大于85%，则评分为0分
    - 如果睡眠效率在75%到84%之间，则评分为1分
    - 如果睡眠效率在65%到74%之间，则评分为2分
    - 如果睡眠效率小于65%，则评分为3分
    '''
    score = round(score, 2)
    if score < 0.65:
        return 3
    elif 0.65 <= score <= 0.74:
        return 2
    elif 0.75 <= score <= 0.84:
        return 1
    elif score > 0.84:
        return 0



In [7]:
def calculate_score_e(score):
    '''
    根据睡眠障碍累加评分计算评分：
    - 如果得分为0，则评分为0分
    - 如果得分在1到9之间，则评分为1分
    - 如果得分在10到18之间，则评分为2分
    - 如果得分大于18，则评分为3分
    '''
    if score == 0:
        return 0
    elif 1 <= score <= 9:
        return 1
    elif 10 <= score <= 18:
        return 2
    elif score > 18:
        return 3


In [8]:
def calculate_score_g(score):
    '''
    根据日间功能障碍得分计算累加评分：
    - 如果得分为0，则评分为0分
    - 如果得分在1到2之间，则评分为1分
    - 如果得分在3到4之间，则评分为2分
    - 如果得分大于4，则评分为3分
    '''
    if score == 0:
        return 0
    elif 1 <= score <= 2:
        return 1
    elif 3 <= score <= 4:
        return 2
    elif score > 4:
        return 3


In [9]:
def load_worksheet_data(path):
    """
    从Excel工作簿的第一个工作表中加载数据。

    参数:
    path (str): Excel文件的路径。

    返回:
    list: 一个包含元组的列表，每个元组代表工作表中的一行。
    """
    # 加载Excel工作簿
    workbook = openpyxl.load_workbook(path)
    
    # 获取活动工作表
    worksheet = workbook.active
    
    # 将工作表中的所有行数据转换为元组列表
    data = list(worksheet.iter_rows(values_only=True))
    
    return data

In [10]:
def psqi_score(path):
    '''
    计算PSQI（匹兹堡睡眠质量指数）各个成分得分以及总得分。
    
    参数:
        path (str): Excel文件的路径，其中包含填写的问卷数据。
    
    返回:
        tuple: 包含三个元素的元组:
            - title_name (list): 包含表格列标题的列表，包括“总分”列。
            - Values (list): 处理后的数据列表，每项数据包括计算后的总分。
            - socore_list (list): 每个人的详细得分信息列表，包括各成分得分和总分。
    '''
    
    # 加载Excel表格
    workbook = openpyxl.load_workbook(path)
    worksheet = workbook.active
    data = list(worksheet.iter_rows(values_only=True))

    socore_list = []
    
    # 导出Excel表格名称处理
    title_name = list(data[0][6:])
    title_name.append(data[0][1])
    last_element = title_name.pop()
    title_name.insert(0, last_element)
    title_name.insert(2, "总分")

    Values = []
    for i in data[1:]:
        
        value = list(i[6:])
        value.append(i[1])
        last_element = value.pop()
        value.insert(0, last_element)

        wenjuan = i[9:]
        
        # 成分A得分计算
        '''
        成分A得分是问卷中第10列的值减1。
        '''
        A = wenjuan[-4] - 1
        
        # 成分B得分计算
        '''
        成分B得分由两个部分组成：
        - b1_1: 问卷中第12列的值，使用 calculate_score_b_1 函数计算得分。
        - b2: 问卷中第15列的值减1，计算总得分后使用 calculate_score_b_2 函数计算最终得分。
        '''
        b1_1 = wenjuan[1]
        b1 = calculate_score_b_1(b1_1)        
        b2 = wenjuan[4] - 1
        b = b1 + b2
        B = calculate_score_b_2(b)

        # 成分C得分计算
        '''
        成分C得分由问卷中第14列的值计算得出，使用 calculate_score_c 函数进行计算。
        '''
        c = wenjuan[3]
        C = calculate_score_c(c)

        # 上床时间和起床时间的处理
        '''
        上床时间和起床时间转换为小时：
        - 计算上床到起床的时间差。
        - 计算睡眠效率。
        '''
        bed = wenjuan[0]
        get = wenjuan[2] 
        sleep_time = wenjuan[3]
        bed = time_to_decimal(bed)
        get = time_to_decimal(get)
        if bed > get:
            d = 24 - bed + get
        else:
            d = get - bed
        efficiency = sleep_time / d
        
        # 成分D得分计算
        D = calculate_score_d(efficiency)
        
        # 成分E得分计算
        e = sum(wenjuan[5:11]) - 6
        E = calculate_score_e(e)

        # 成分F得分计算
        F = wenjuan[-3] - 1
        
        # 成分G得分计算
        g1 = wenjuan[-2] - 1
        g2 = wenjuan[-1] - 1
        g = g1 + g2
        G = calculate_score_g(g)

        # PSQI总得分
        socore = A + B + C + D + E + F + G
        
        # 得分字典列表
        socore_list.append({
            "姓名": i[6],
            "填写日期": i[1],
            "成分A：睡眠质量得分": A,
            "成分B：睡眠潜伏期得分": B,
            "成分C：睡眠持续时间得分": C,
            "成分D：习惯性睡眠效率得分": D,
            "成分E：睡眠障碍得分": E,
            "成分F：睡眠药物使用得分": F,
            "成分G：日间功能障碍得分": G,
            "总分": socore
        })
        
        # 原始表格处理部分插入总分
        value.insert(2, socore)
        
        # 原始表格处理分析数值列表
        Values.append(value)

    return title_name, Values, socore_list

In [11]:
def modify_data(data):
    '''
    将数据中的数值转换成文字描述范围。

    参数:
    - data: 包含需要转换的数值的字典

    返回:
    - 更新后的字典，包含文字描述的数值
    '''
    frequency_mapping = {
        1: "无，计分0",
        2: "<1次/周，计分1",
        3: "<2次/周，计分2",
        4: ">=3次/周，计分3"
    }
        
    quality_mapping = {
        1: "很好，计分0",
        2: "较好，计分1",
        3: "较差，计分2",
        4: "很差，计分3"
    }

    energy_mapping = {
        1: "没有，计分0",
        2: "偶尔有，计分1",
        3: "有时有，计分2",
        4: "经常，计分3"
    }

    # 转换频率数据
    for key in ['9、近一个月，您常感到困倦吗—', '6、近一个月，您是否因下列情况影响睡眠而烦恼—A.入睡困难（30分钟内不能入睡）', '6、B.夜间易醒或早醒', '6、C.夜间去厕所', 
                '6、D.感觉呼吸不畅或憋气', '6、E.咳嗽或鼾声高', '6、F.感到太冷', '6、G.感到太热', '6、H.做噩梦', '6、I.疼痛不适', '6、J.其他影响睡眠的事情（如病痛、环境等）', '8、近一个月，您用药物催眠的情况—']:
        data[key] = frequency_mapping[data[key]]
 
    # 转换质量数据
    data["7、近一个月，总的来说，您认为自己的睡眠质量—"] = quality_mapping[data["7、近一个月，总的来说，您认为自己的睡眠质量—"]]
    
    # 转换能量数据
    data["10、近一个月您是否精力不足？做事力不从心？—"] = energy_mapping[data["10、近一个月您是否精力不足？做事力不从心？—"]]

    # 转换实际睡眠时间
    sleep_duration = data["5、近一个月，您每夜通常实际睡眠__小时(不等于卧床时间)"]
    if sleep_duration < 5:
        data["5、近一个月，您每夜通常实际睡眠__小时(不等于卧床时间)"] = "<5h"
    elif 5 <= sleep_duration <= 6:
        data["5、近一个月，您每夜通常实际睡眠__小时(不等于卧床时间)"] = "5～6h"
    elif 6 < sleep_duration <= 7:
        data["5、近一个月，您每夜通常实际睡眠__小时(不等于卧床时间)"] = "6～7h"
    else:
        data["5、近一个月，您每夜通常实际睡眠__小时(不等于卧床时间)"] = ">7h"
    
    # 转换入睡等待时间
    wait_time = data["3、近一个月，您从上床到入睡通常需要__分钟"]
    if wait_time <= 15:
        data["3、近一个月，您从上床到入睡通常需要__分钟"] = "<=15min"
    elif 16 <= wait_time <= 30:
        data["3、近一个月，您从上床到入睡通常需要__分钟"] = "16～30min"
    elif 31 <= wait_time <= 60:
        data["3、近一个月，您从上床到入睡通常需要__分钟"] = "31～60min"
    else:
        data["3、近一个月，您从上床到入睡通常需要__分钟"] = ">=60min"
    
    return data


In [12]:
def expand_cell(text):
    '''
    设置最大单元格的大小。
    
    Excel单元格的最大字符数限制为32767个字符。如果输入文本超过此限制，则截断为32767个字符。

    参数:
        text (str): 需要检查并可能截断的文本。

    返回:
        str: 截断后的文本（如果超过32767字符），或原始文本（如果不超过32767字符）。
    '''
    # 检查文本是否为字符串，并且长度是否超过32767字符
    if isinstance(text, str) and len(text) > 32767:  
        # 如果超过限制，则截断文本为前32767字符
        return text[:32767]
    # 如果不超过限制，则返回原始文本
    return text


In [13]:
def Resize_Save(df, excel_path):
    '''
    调整Excel表格行列间距大小并保存为Excel文件。

    参数:
    - df: 需要调整的DataFrame
    - excel_path: 保存的Excel文件路径

    操作:
    - 确保日期格式正确
    - 处理单元格内容长度
    - 保存为Excel文件
    - 调整列宽
    '''
    # 确保日期格式正确
    df['提交答卷时间'] = df['提交答卷时间'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # 处理单元格内容长度
    for col in df.columns:
        df[col] = df[col].apply(expand_cell)

    # 保存为Excel文件并调整列宽
    excel_file_path = excel_path
    with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
        df.to_excel(writer, index=False)
        worksheet = writer.sheets['Sheet1']

        # 调整列宽
        for idx, col in enumerate(worksheet.columns):
            max_length = 0
            col_name = col[0].column_letter  # 获取列名
            for cell in col:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2) * 1.7  # 调整宽度，放大1.7倍
            worksheet.column_dimensions[col_name].width = adjusted_width

        # 额外调整标题的列宽
        for idx, col in enumerate(df.columns):
            max_length = len(col)  # 标题长度
            col_letter = openpyxl.utils.get_column_letter(idx + 1)
            # 放大标题宽度
            worksheet.column_dimensions[col_letter].width = max(max_length * 1.5, worksheet.column_dimensions[col_letter].width)

In [14]:
def Group_Anylise(keys, entries, excel_path):
    '''

    保存计算后PSQI详细输出结果
    分析问卷数据并判断是否满足入组条件，然后保存结果。

    参数:
    - keys: 包含表头的列表
    - entries: 包含数据行的列表
    - excel_path: 保存计算后结果的Excel文件路径

    操作:
    - 将数值转换成文字范围
    - 转换为DataFrame并排序
    - 保存计算后PSQI详细输出结果
    - 统计满足入组条件的对象并保存
    - 统计需要回访的对象并保存
    '''
    # 将表格中的数值转换成范围
    data_dict_list = [dict(zip(keys, entry)) for entry in entries]
    for data in data_dict_list:
        modify_data(data)
    
    # 转换成DataFrame形式
    df = pd.DataFrame(data_dict_list)
    
    # 将时间转换成datetime类型
    df['提交答卷时间'] = pd.to_datetime(df['提交答卷时间'])
    
    # 按照问卷提交时间和姓名重排序
    df = df.sort_values(by=['提交答卷时间', '1、个人基本信息—姓名：'])

    # 统计满足入组条件的对象
    # 假设 psqi得分>=7 表示满足条件
    df_filtered = df[df['总分'] >= 7]
    # 假设 "入睡时间>=3次/周，计分3" 存储在名为 '入睡频率' 的列中，并且其值为 3 表示满足条件
    df_filtered = df_filtered[df_filtered['6、近一个月，您是否因下列情况影响睡眠而烦恼—A.入睡困难（30分钟内不能入睡）'] == ">=3次/周，计分3"]
    # 假设 "从上床到入睡通常需要__分钟 >=60min" 存储在名为 '入睡时间' 的列中，并且其值>=60表示满足条件
    df_filtered = df_filtered[df_filtered['3、近一个月，您从上床到入睡通常需要__分钟'].isin([">=60min", "30～60min"])]

    # 统计psqi小于7分但满足其他两个入睡困难条件的对象, 保存以便进行回访
    df_f = df[df['总分'] < 7]
    df_f = df_f[df_f['6、近一个月，您是否因下列情况影响睡眠而烦恼—A.入睡困难（30分钟内不能入睡）'] == ">=3次/周，计分3"]
    df_f = df_f[df_f['3、近一个月，您从上床到入睡通常需要__分钟'].isin([">=60min", "30～60min"])]

    # 保存计算后PSQI详细输出结果
    Resize_Save(df, excel_path)

    # 保存满足条件需要入组的对象
    # Resize_Save(df_filtered, "../result/入组名单.xlsx")
    
    # 保存可能需要回访的对象
    # Resize_Save(df_f, "../result/回访名单.xlsx")

In [15]:
def Restore_Save_Excel(data, excel_path):
    '''
    将计算后的PSQI各成分得分保存为Excel表格，并进行排序和格式调整。

    参数:
    - data: 包含计算结果的字典列表
    - excel_path: 保存的Excel文件路径

    操作:
    - 将日期格式统一为字符串
    - 将数据转换为DataFrame
    - 按姓名和填写日期排序
    - 添加填写次数
    - 调整列宽
    '''
    # 将填写日期统一为字符串格式
    for entry in data:
        if isinstance(entry.get('填写日期'), datetime):
            entry['填写日期'] = entry['填写日期'].strftime('%Y/%m/%d %H:%M:%S')

    # 转换为DataFrame
    df = pd.DataFrame(data)

    # 获取姓名的首次出现顺序
    name_order = {name: i for i, name in enumerate(df['姓名'].unique())}

    # 根据姓名的顺序映射表进行排序，并在每个人名组内按填写日期排序
    df['姓名顺序'] = df['姓名'].map(name_order)
    df_sorted = df.sort_values(by=['姓名顺序', '填写日期']).drop(columns=['姓名顺序']).reset_index(drop=True)

    # 添加填写次数
    df_sorted['填写次数'] = df_sorted.groupby('姓名').cumcount() + 1

    # 将总分放在指定列位置
    columns_order = ['姓名', '填写日期', '填写次数', '成分A：睡眠质量得分', '成分B：睡眠潜伏期得分', '成分C：睡眠持续时间得分', '成分D：习惯性睡眠效率得分', '成分E：睡眠障碍得分', '成分F：睡眠药物使用得分', '成分G：日间功能障碍得分', '总分']
    df_sorted = df_sorted[columns_order]

    # 保存为Excel文件并调整列宽
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        df_sorted.to_excel(writer, index=False, sheet_name='Sheet1')
        worksheet = writer.sheets['Sheet1']
        
        # 调整列宽
        for column_cells in worksheet.columns:
            max_length = 0
            column = column_cells[0].column_letter
            for cell in column_cells:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except Exception as e:
                    print(f"Error while processing cell: {e}")
            adjusted_width = (max_length + 2) * 1.5
            worksheet.column_dimensions[column].width = adjusted_width


In [16]:
if __name__ =="__main__":
    path = '../data/修改前psqi问卷.xlsx'
    Title_Name,Values,Socore_List = psqi_score(path)
    Group_Anylise(Title_Name,Values,excel_path="../result/修改前PSQI详细输出结果表.xlsx")
    Restore_Save_Excel(Socore_List,"../result/修改前PSQI各成分得分表.xlsx")
  
  